In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

In [2]:
#prepare transform

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

In [3]:
# prepare datasets

BATCH_SIZE = 50

train_datasets = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    transform=transform,
    download=True,
)

train_loader = torch.utils.data.DataLoader(
    train_datasets,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

In [4]:
test_datasets = torchvision.datasets.CIFAR10(
    root="../data",
    train=False,  # ✅ テストデータにする
    transform=transform,
    download=True,
)

test_loader = torch.utils.data.DataLoader(
    test_datasets,
    batch_size=BATCH_SIZE,
    shuffle=False,  # テストなのでシャッフル不要
    num_workers=2
)

In [5]:
#make CNN model

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # 修正: 64*8*8に変更
        self.fc2 = nn.Linear(128, 10)  # 10クラス分類
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # ✅ conv1 → ReLU → MaxPool
        x = self.pool(F.relu(self.conv2(x)))  # ✅ conv2 → ReLU → MaxPool

        x = x.view(x.size(0), -1)  # ✅ Flatten（batch_size, 64 * 8 * 8）
        
        x = F.relu(self.fc1(x))  # ✅ FC1 → ReLU
        x = self.fc2(x)  # ✅ FC2（出力層）

        return x

In [6]:
model = CNN()
criterion = nn.CrossEntropyLoss()  # 損失関数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 最適化手法


In [7]:
#評価関数

def eval(model, test_loader, writer, epoch, device="cpu"):
    model.to(device)
    model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    writer.add_scalar("Accuracy/test", accuracy, epoch)
    return accuracy

In [ ]:
#define training function
from torch.utils.tensorboard import SummaryWriter

def train(model, trainloader, criterion, optimizer, num_epochs = 5, device = "cpu"):
    model.to(device)
    writer = SummaryWriter(log_dir="runs/cnn_exp_2025-02-28-epoch=10")
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total if total > 0 else 0

        writer.add_scalar("Loss/train", epoch_loss, epoch)
        writer.add_scalar("Accuracy/train", epoch_acc, epoch)

        # 🔥 各エポックごとに `evaluate()` を呼び出す！
        test_acc = eval(model, test_loader, writer, epoch, device)

        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.2%} - Test Acc: {test_acc:.2f}%")

    writer.close()
    print("Training Finished")

In [9]:

train(model, train_loader, criterion, optimizer, num_epochs=10 )

Test Accuracy: 59.00%
Epoch 1/10 - Loss: 1.4486 - Train Acc: 47.80% - Test Acc: 59.00%
Test Accuracy: 64.45%
Epoch 2/10 - Loss: 1.0929 - Train Acc: 61.30% - Test Acc: 64.45%
Test Accuracy: 65.20%
Epoch 3/10 - Loss: 0.9479 - Train Acc: 66.46% - Test Acc: 65.20%
Test Accuracy: 68.75%
Epoch 4/10 - Loss: 0.8553 - Train Acc: 70.04% - Test Acc: 68.75%
Test Accuracy: 70.01%
Epoch 5/10 - Loss: 0.7724 - Train Acc: 72.83% - Test Acc: 70.01%
Test Accuracy: 68.52%
Epoch 6/10 - Loss: 0.7052 - Train Acc: 75.15% - Test Acc: 68.52%
Test Accuracy: 70.64%
Epoch 7/10 - Loss: 0.6437 - Train Acc: 77.35% - Test Acc: 70.64%
Test Accuracy: 70.33%
Epoch 8/10 - Loss: 0.5898 - Train Acc: 79.17% - Test Acc: 70.33%
Test Accuracy: 70.75%
Epoch 9/10 - Loss: 0.5323 - Train Acc: 81.34% - Test Acc: 70.75%
Test Accuracy: 70.35%
Epoch 10/10 - Loss: 0.4786 - Train Acc: 83.09% - Test Acc: 70.35%
